# Домашнее задание 1

**Выполнил:** Дробышевский Илья Александрович

**На сколько выполнил:** 7.35$⇐$(10.5-30%)

**Правила игры:**

* возле каждой задачи указано число баллов (в сококупности можно получить 10 баллов)

* дополнительные задачи выделены звездочкой (также указано число баллов). Данные баллы можно будет использовать для улучшения результата за любое домашнее задание

* все подсчеты необходимо делать с помощью pandas-numpy (использовать как можно меньше циклов, за использование циклов, где это не требуются, будем штрафовать)

* для визуализации можно использовать matplotlib-seaborn-plotly (учтите, что все графики должны выглядеть опрятно, иметь название-подписи, за нечитабельные графики будем штрафовать)

* возможно, что в данных есть пустоты и невалидные значения. Перед началом любой работы необходимо исследовать, и если такие есть, убрать их

* соблюдайте чистоту кода: переменные должны быть осмысленными, не повторяйтесь, если в этом нет необходимости (иначе используйте функции). Чем чище код - тем проще нам поставить вам хорошую оценку :)

* общая рекомендация: на вопросы со звездочкой отвечать в самом конце, возможно, будет проще)

## Данные 

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут: (будет ссылка)

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с бразильского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем, пожалуй!

In [2]:
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.express as px
from dateutil.parser import parse
import random

order_items = pd.read_csv(
    '/content/olist_order_items_dataset.csv')
orders = pd.read_csv(
    '/content/olist_orders_dataset.csv')
order_reviews = pd.read_csv('/content/olist_order_reviews_dataset.csv'
    )
order_payments = pd.read_csv(
    '/content/olist_order_payments_dataset.csv')
products = pd.read_csv('/content/olist_products_dataset.csv')
sellers = pd.read_csv('/content/olist_sellers_dataset.csv')
order_customer = pd.read_csv(
    '/content/olist_customers_dataset.csv')
geolocation = pd.read_csv(
    '/content/olist_geolocation_dataset.csv')
english_categories = pd.read_csv(
    '/content/product_category_name_translation.csv')

## Задание 1 (0.4 балла)

Определите:

* Число товаров
* Среднюю стоимость товара

в разрезе категорий (все категории должны быть на английском языке)

In [ ]:
products = pd.merge(english_categories, products)

In [ ]:
fig = px.bar(products.groupby(['product_category_name_english']).agg({'product_id': 'count'}).reset_index().rename(
    columns={'product_category_name_english': 'categories', 'product_id': 'quantity'}).sort_values('quantity'),
    x='quantity', y='categories', color='categories', text_auto=True,
    title="Number of products", width=1400, height=750)
fig.update_traces(textposition="outside", cliponaxis=False)
fig.update_yaxes(tickfont={'size': 7})#чтобы все категории влезли, уменьшим размер текста
fig.show()

In [ ]:
fig = px.bar(pd.merge(products, order_items[['price', 'product_id']], on=['product_id']).groupby(
    ['product_category_name_english']).agg({'price': 'mean'}).reset_index().rename(
    columns={'product_category_name_english': 'categories'}).sort_values('price'), x='price',
    y='categories', text_auto=True, color='categories', title='Product average cost', width=1400, height=750)
fig.update_traces(textposition="outside", cliponaxis=False)
fig.update_yaxes(tickfont={'size': 7})
fig.show()

## Задание 2 (1,5 балла)

Определите для каждого продавца основную категорию их продаж.

Отобразите категории по числу продавцов, для которых данная категория основная, а также создайте график, по которому можно выбрать отдельно штат и посмотреть такое же распределение



In [ ]:
#ищем самую часту по продажам категорию для каждого селлера

seller_category = pd.merge(products, pd.merge(order_items, sellers, on=['seller_id']), on=['product_id']).groupby(['seller_id']).agg(
    pd.Series.mode)['product_category_name_english'].reset_index()
# seller_category

In [ ]:
#теперь как бы меняем всё местами и находим количество селлеров для каждой категории

category_sel_count = seller_category.product_category_name_english.apply(pd.Series).merge(seller_category,
                                                                                          right_index=True,
                                                                                          left_index=True).drop(
    ["product_category_name_english"], axis=1).melt(id_vars=['seller_id'],
                                                    value_name="product_category_name_english").drop("variable",
                                                                                                     axis=1).dropna().merge(
    sellers[['seller_state', 'seller_id']])
# category_sel_count.groupby(['product_category_name_english'])['seller_id'].count().reset_index()

In [ ]:
#делаем разбивку на штаты: для этого проходимся по всем штатам и мёржим имеющуюся таблицу с колонкой

states = list(sellers.seller_state.unique())
category_per_state = category_sel_count[category_sel_count['seller_state'] == states[0]].groupby(['product_category_name_english'])[
    'seller_id'].count().reset_index().rename(
    columns={'seller_id': states[0]})
# category_per_state

for i, state in enumerate(states):
    if i != 0:
        category_per_state = pd.merge(category_per_state,
                                      category_sel_count[category_sel_count['seller_state'] == state].groupby(
                                          ['product_category_name_english'])[
                                          'seller_id'].count().reset_index().rename(
                                          columns={'seller_id': state}), how='outer')
category_per_state = category_per_state.fillna(0)
# category_per_state

In [ ]:
fig = px.bar(category_per_state.sort_values(states), x=states, y='product_category_name_english', title='Categories in each state',
             width=1250, height=750)
fig.update_xaxes(title_text='sellers quantity')
fig.update_yaxes(title_text='categories', tickfont={'size': 7})
fig.show()

## Задание 3 (1,5 балл)

Покажите процент

* суммы покупок в деньгах

* суммы покупок в штуках

по категории для доставленных заказов, а также разбивку по штатам

In [ ]:
order_it_products_wo_states=pd.merge(pd.merge(order_items, orders), products).drop(
    ['product_id','seller_id', 'shipping_limit_date', 'customer_id', 'order_purchase_timestamp',
     'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date',
     'order_estimated_delivery_date', 'product_category_name', 'product_name_lenght', 'product_description_lenght',
     'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'], axis=1)

order_it_products_wo_states['costs']=order_it_products_wo_states['price']+order_it_products_wo_states['freight_value']

percents_wo_states = order_it_products_wo_states[order_it_products_wo_states['order_status'] == 'delivered'].groupby(
    'product_category_name_english').agg(
    {'costs': 'sum', 'order_item_id': 'count'}).reset_index().rename(
    columns={'costs': 'price_deliv', 'order_item_id': 'order_item_id_deliv'})


percents_wo_states['price_percent'] = percents_wo_states['price_deliv'] / percents_wo_states['price_deliv'].sum() * 100
percents_wo_states['order_item_id_percent'] = percents_wo_states['order_item_id_deliv'] / percents_wo_states[
    'order_item_id_deliv'].sum() * 100

In [ ]:
order_it_products = pd.merge(pd.merge(pd.merge(orders,order_customer[['customer_id','customer_unique_id','customer_state']]), order_items),
                             products).drop(
    ['customer_id','order_id', 'seller_id', 'shipping_limit_date', 'customer_id', 'order_purchase_timestamp',
     'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date',
     'order_estimated_delivery_date', 'product_category_name', 'product_name_lenght', 'product_description_lenght',
     'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'], axis=1)
order_it_products['costs']=order_it_products['price']+order_it_products['freight_value']

percents = order_it_products[
        (order_it_products['order_status'] == 'delivered') & (order_it_products['customer_state'] == states[0])].groupby(
        'product_category_name_english').agg(
        {'costs': 'sum', 'order_item_id': 'sum'}).reset_index().rename(
        columns={'costs': states[0] + '_price_deliv', 'order_item_id': states[0] + '_item_deliv'})

percents[states[0] + '_price_percent'] = percents[states[0] + '_price_deliv'] / percents[states[0] + '_price_deliv'].sum() * 100
percents[states[0] + '_items_percent'] = percents[states[0] + '_item_deliv'] / percents[states[0] + '_item_deliv'].sum() * 100
percents = percents.drop([states[0] + '_price_deliv', states[0] + '_item_deliv'], axis=1)

for i, state in enumerate(states):
    if i != 0:
        percents = pd.merge(percents,order_it_products[
                (order_it_products['order_status'] == 'delivered') & (
                        order_it_products['customer_state'] == state)].groupby(
                'product_category_name_english').agg(
                {'costs': 'sum', 'order_item_id': 'sum'}).reset_index().rename(
                columns={'costs': state + '_price_deliv', 'order_item_id': state + '_item_deliv'}), how='outer')
        percents[state + '_price_percent'] = percents[state + '_price_deliv'] / percents[state + '_price_deliv'].sum() * 100
        percents[state + '_items_percent'] = percents[state + '_item_deliv'] / percents[state + '_item_deliv'].sum() * 100
        percents = percents.drop(
            [state + '_price_deliv', state + '_item_deliv'], axis=1)
percents = percents.fillna(0)


fig = px.bar(percents.sort_values([i + '_price_percent' for i in states]), x=[i + '_price_percent' for i in states], y='product_category_name_english',
             title='Price percents in each state', width=1250, height=750)
fig.update_xaxes(title_text='price percent')
fig.update_yaxes(title_text='categories', tickfont={'size': 7})
fig.show()

In [ ]:
fig = px.bar(percents.sort_values([i + '_items_percent' for i in states]), x=[i + '_items_percent' for i in states], y='product_category_name_english',
             title='Items percents in each state', width=1250, height=750)
fig.update_xaxes(title_text='items percent')
fig.update_yaxes(title_text='categories', tickfont={'size': 7})
fig.show()

## Задание 4 (0.6 балла)

Визуализируйте зависимость между средним числом покупок и:

* числом фотографий товара

* кол-во символов в описании товара (аггрегируйте с шагом 20)

* кол-во символов в названии (аггрегируйте с шагом 5)

In [ ]:
prod_order_it = pd.merge(
    products[['product_id', 'product_photos_qty', 'product_description_lenght', 'product_name_lenght']],
    order_items[['order_id', 'product_id']])

purchase_count=pd.merge(prod_order_it.drop('order_id', axis=1), prod_order_it.groupby('product_id')['order_id'].count().reset_index())

fig=px.bar(purchase_count.groupby('product_photos_qty')['order_id'].mean().reset_index().rename(columns={'order_id':'average_purchases_quantity'}),x='product_photos_qty',y='average_purchases_quantity',color='average_purchases_quantity',title='Quantity of purchases to a number of photos')
fig.update_xaxes(title_text='photos quantity')
fig.update_yaxes(title_text='average purchases quantity')

In [ ]:
purchase_count['product_description_lenght']=purchase_count['product_description_lenght'].apply(lambda x:(x // 20) * 20)



fig=px.bar(purchase_count.groupby('product_description_lenght')['order_id'].mean().reset_index().rename(columns={'order_id':'average_purchases_quantity'}),x='product_description_lenght',y='average_purchases_quantity',color='average_purchases_quantity',title='Quantity of purchases to a desciption length')
fig.update_yaxes(title_text='average purchases quantity')
fig.update_xaxes(title_text='description length')
fig.show()

In [ ]:
purchase_count['product_name_lenght']=purchase_count['product_name_lenght'].apply(lambda x:(x // 5) * 5)



fig=px.bar(purchase_count.groupby('product_name_lenght')['order_id'].mean().reset_index().rename(columns={'order_id':'average_purchases_quantity'}),x='product_name_lenght',y='average_purchases_quantity',color='average_purchases_quantity',title='Quantity of purchases to a name length')
fig.update_xaxes(title_text='name length')
fig.update_yaxes(title_text='average purchases quantity')

## Задание 5* (1 балл)

Имея полученные данные по товарам, чекам и категориям, а также сделав дополнительные расчеты (если необходимо), предположите:

1. Продавцов каких категорий необходимо привлекать в маркетплейс?

2. Какие категории в каких штатах необходимо развивать?

3. Развитие каких категорий не принесут большого эффекта и почему?

4. Какие рекомендации можно дать для продавцов, чтобы их товар лучше продавался?



---


1. Если маркетплэйс, как и любой другой бизнес, хочет приносить как можно больше денег, то логично приглашать не только очень дорогие категории, как например computers, но и очень продававемые, такие как bed_beth_table, sport_leisure, furniture_decor, health_beaty, housewares; хоть эти товары нельзя назвать жизненнонеобходимыми, но люди крайне от них зависят.

2. Видно, что штатах, которые не так густо населены, как Сан-Паулу и Рио, очень многие категории попросту не поступают в продажу. Поэтому можно попытаться развить их. Ну а вообще, возможно, в этих штатах у людей другие интересы и они не хотят покупать эти категории.

3. хз

4. В объявлении должно быть около 6 фотографий 30-55 символов в названии.

## Задание 6 (0.5 балла)

Определите средний чек покупки (добавьте разбивку на стоимость самого заказ и стоимость доставки) и среднее число товаров в заказе

In [ ]:
print(
    f"Average bill is {(order_items['price'] + order_items['freight_value']).mean()}\naverage order price is {order_items['price'].mean()}\naverage freight value is {order_items['freight_value'].mean()}\naverage order items is {order_items.groupby('order_id').max()['order_item_id'].reset_index()['order_item_id'].mean()}")

Average bill is 140.6440589436307
average order price is 120.65373901464716
average freight value is 19.990319928983578
average order items is 1.1417306873695092


## Задание 7 (0.2 балла)

Определите среднее число покупок на пользователя (обратите внимание на идентификаторы)

In [ ]:
customer_orders = pd.merge(pd.merge(orders[['order_id', 'customer_id']],order_items[['order_item_id','order_id']]), order_customer[['customer_id', 'customer_unique_id']]).drop(
    ['customer_id'], axis=1)
customer_orders=pd.merge(customer_orders.drop('order_item_id',axis=1),customer_orders.groupby('order_id').max()['order_item_id'].reset_index()).dropna()

print(f"Average number of purchase is {customer_orders.order_item_id.sum() / customer_orders.customer_unique_id.count()}")

Average number of purchase is 1.3956679982245894


## Задание 8 (0.2 балла)

Отобразите среднюю сумму оплаты по типу оплаты

In [ ]:
order_payments.payment_type.unique()

array(['credit_card', 'boleto', 'voucher', 'debit_card', 'not_defined'],
      dtype=object)

Видим тут какого-то шпиона not_defined. Предалагаю его выкинуть:

In [ ]:
lst = []
for pay_type in list(order_payments.payment_type.unique()):
  if pay_type!='not_defined':
    lst.append((order_payments[order_payments['payment_type'] == pay_type]['payment_value'].mean(),pay_type))

fig=px.bar(pd.DataFrame(lst,columns={'payment_type','average_sum'}),\
           x='payment_type',y='average_sum', width=1250, \
           height=500,title='Average sum in each purchase method',\
           color='average_sum')
fig.update_xaxes(title_text='payment type')
fig.update_yaxes(title_text='average sum')
fig.show()

## Задание 9 (0.4 балла)

Обратите внимание на число платежей (payment_installments). Выясните, какая сумма оплаты указана: за целую покупку или только за ее часть?

Также покажите средний чек в зависимости от числа оплаты. Есть ли какая-то связь между ними?



---



Возьмём какой-то рандомный заказ, у которого payment_installments больше 1. Например это будет b81ef226f3fe1789b1e8b2acac839d17.

In [ ]:
order_payments[order_payments['order_id'] == 'b81ef226f3fe1789b1e8b2acac839d17']

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33


Видим, что чел взял его а-ля в рассрочку на 8 месяцев и либо платит платит каждый месяц 99.93 у.е., либо это была полная сумма.

In [ ]:
order_items[order_items['order_id']=='b81ef226f3fe1789b1e8b2acac839d17']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
80985,b81ef226f3fe1789b1e8b2acac839d17,1,af74cc53dcffc8384b29e7abfa41902b,213b25e6f54661939f11710a6fddb871,2018-05-02 22:15:09,79.8,19.53


Ну и теперь становится понятно, что это была полная цена, а не ежемесячная выплата.

In [ ]:
fig=px.bar(order_payments.groupby('payment_sequential')['payment_value'].mean().reset_index(),x='payment_sequential',y='payment_value',title='Average bill for each payment_sequential')
fig.show()

## Задание 10 (0.4 балла)

Визуализируйте зависимость между ценой заказа и ценой доставки по штатам покупки. Есть ли какая-нибудь зависимость и как вы можете ее обосновать?

In [ ]:
price_and_freight=pd.merge(pd.merge(order_items[['order_id','price','freight_value']],orders[['order_id','customer_id']]),order_customer[['customer_id','customer_state']])
price_and_freight['costs']=price_and_freight['price']+price_and_freight['freight_value']
price_and_freight['percentage']=price_and_freight['freight_value']/price_and_freight['costs']
fig=px.bar(price_and_freight.groupby('customer_state').agg({'percentage':'mean','costs':'mean'}).reset_index().rename(columns={'customer_state':'states'}).sort_values('percentage'),x='percentage',y='states',color='costs',width=1250,height=750,title='Percentage of delivery from the price by state')
fig.show()

## Задание 11 (0.4 балла)

Выведите график по числу покупок (в разбивке по статусам) и временем покупки (аггрегируйте покупки до дня)

In [ ]:
purch_count = pd.merge(orders[['order_id', 'order_status', 'order_approved_at']], order_items[['order_id', 'order_item_id']])
purch_count = purch_count.drop(purch_count[purch_count['order_status'] == 'canceled'].index)
purch_count = pd.merge(purch_count.groupby('order_id').max()['order_item_id'].reset_index(), purch_count).dropna()
purch_count['order_approved_at'] = purch_count['order_approved_at'].apply(lambda x: parse(x))
purch_count['day'] = purch_count['order_approved_at'].apply(lambda x: x.day)
statuses = list(purch_count['order_status'].unique())
days_purchases = purch_count[purch_count['order_status'] == statuses[0]].groupby('day')['order_item_id'].sum().reset_index().rename(
    columns={'order_item_id': str(statuses[0])})
for i, status in enumerate(statuses):
    if i != 0:
        days_purchases = pd.merge(days_purchases, purch_count[purch_count['order_status'] == status].groupby('day')['order_item_id'].sum().reset_index().rename(
            columns={'order_item_id': str(status)}), how='outer')

fig = px.bar(days_purchases.fillna(0), x='day', y=statuses, title='Purchases quantity by status in different days')
fig.update_yaxes(title_text='purchases quantity')
fig.show()

## Задание 12* (1 балл)

Используя полученные данные, а также сделав дополнительные вычисления (если необходимо), предположите:

1. Есть ли сезонность в покупках на маркетплейсе?

2. Есть ли какие-то необъяснимые падение/рост продаж? С чем это может быть связано?



---


2. Всё, что я вижу, это то, что к концу месяца покупательская способность падает.

## Задание 13 (0.3 балла)

Посчитайте CSAT (customer satisfaction - средняя оценка ревью) и отобразите средний CSAT по дням



In [ ]:
order_reviews['review_creation_date']=order_reviews['review_creation_date'].apply(lambda x:parse(x))
order_reviews['day']=order_reviews['review_creation_date'].apply(lambda x:x.day)
fig=px.bar(order_reviews.groupby(['day'])['review_score'].mean().reset_index(),x='day',y='review_score',color='review_score',title='CSAT in each day')
fig.update_yaxes(title_text='CSAT')
fig.show()

## Задание 14 (0.3 балла)

Найдите топ-5 лучших и топ-5 худших продавцов по средней оценке и посчитайте их число продаж

In [ ]:
top5_best = pd.merge(order_reviews, order_items[['order_id', 'seller_id']]).groupby('seller_id')[
                'review_score'].mean().reset_index().sort_values('review_score', ascending=False)['seller_id'][:5]
top5_worst = pd.merge(order_reviews, order_items[['order_id', 'seller_id']]).groupby('seller_id')[
                 'review_score'].mean().reset_index().sort_values('review_score', ascending=False)['seller_id'][-5:]

sale_amount_best = pd.merge(top5_best, order_items).drop_duplicates().groupby('seller_id')[
    'product_id'].count().reset_index().rename(columns={'product_id': 'quantity'})
sale_amount_worst = pd.merge(top5_worst, order_items).drop_duplicates().groupby('seller_id')[
    'product_id'].count().reset_index().rename(columns={'product_id': 'quantity'})
pd.merge(top5_worst, order_items)

print('This is the best sellers:' + '\n', sale_amount_best)
print()
print('-'*50)
print()
print('This is the worst sellers:' + '\n', sale_amount_worst)

This is the best sellers:
                           seller_id  quantity
0  1cd9e0cc1839d55516843def5600816d         5
1  702835e4b785b67a084280efca355756         1
2  9fbd5666d8618fa571c4230fafa09bd6         1
3  a20d8058c866dbaeca014d2cd20ac62e         2
4  d3891911c2feae53c52fea2bd14054e5         2

--------------------------------------------------

This is the worst sellers:
                           seller_id  quantity
0  1f7dfad2cb384ea4d2d7e1ffbd78c407         1
1  1fa2d3def6adfa70e58c276bb64fe5bb         1
2  88af55b4a7ca402b27df16f7c7c9b5d2         1
3  913fd0ebba70bb919f7254a02239173c         1
4  92992bec39dde5766f590d3b1f46d465         1


## Задание 15 (0.3 балла)

Посмотрите, как быстро отвечают пользователи (сделайте аггреграцию по дням) и влияет ли это на оценку?

In [ ]:
order_reviews['review_answer_timestamp']=order_reviews['review_answer_timestamp'].apply(lambda x: parse(x))
answer_and_time = pd.merge(order_reviews[['review_score', 'review_answer_timestamp', 'review_creation_date', 'order_id']],pd.merge(orders[['order_id','customer_id']],order_customer[['customer_id','customer_unique_id']])).drop(['order_id','customer_id'],axis=1)
answer_and_time['days'] = answer_and_time['review_answer_timestamp'] - answer_and_time['review_creation_date']



answer_and_time['days'] = answer_and_time['days'].apply(lambda x:(x.days//2)*2)#агрегируем по два дня
fig = px.bar(answer_and_time.groupby('days').agg({'customer_unique_id': 'nunique', 'review_score': 'mean'}).reset_index(),
             x='days', y='customer_unique_id', color='review_score', title='Speed of answer')
fig.update_yaxes(title_text='customers')
fig.update_xaxes(range=[-1,30])  #дальше смотреть не очень хочется, поэтому сделаем такой масштаб.
fig.show()

## Задание 16 (0.5 балла)

Отобразите распределение оценок для заказов, которые были доставлены вовремя и которые пришли с опозданием. Насколько влияет факт опоздания на оценку?

In [ ]:
#создадим столбец, где будет true, если опоздал, ну и false иначе. для этого просто проверим, что разница между прибытием и заявленным 
#дедлайном не отрицательная

package_is_late = pd.merge(orders[['order_id', 'order_estimated_delivery_date', 'order_delivered_customer_date']],
                           order_reviews[['order_id', 'review_score']]).dropna()
package_is_late['order_estimated_delivery_date'] = package_is_late['order_estimated_delivery_date'].apply(
    lambda x: parse(x))
package_is_late['order_delivered_customer_date'] = package_is_late['order_delivered_customer_date'].apply(
    lambda x: parse(x))
package_is_late['is_late'] = (package_is_late['order_estimated_delivery_date'] - package_is_late[
    'order_delivered_customer_date']).apply(lambda x: x.days < 0)
fig=px.bar(package_is_late.groupby(['review_score','is_late'])['order_id'].count().reset_index(),x='review_score',y='order_id',color='is_late',title='Score distribution')
fig.update_yaxes(title_text='orders quantity')
fig.show()

Исходя из графика видно, что товары, которые опоздали, чаще всего получают оценку 1(но в то же время следующая по числу оценка 5). В то же время товары, которые пришли без опоздания имеют оценку 5 более чем в половине случае.

## Задание 17* (1,5 балла)

Есть ли среди пользователей хейтеры? Определите их и вычислите средний CSAT без их участия



---


Могу описать примерный алгоритм их поиска: по идее у хейторов должно быть 
много комментариев с низкой оценкой и делать они это будут с разных акков, поэтому надо смотреть на уникальные айдишники покупателей, количество из покупок и оценки; если средняя оценка стремится к 0, то мы нашли хейтера.

## Задание 18 (0.4 балла)

Изобразите зависимость между ценой заказа и числом дней между покупкой и доставкой с разбивкой по признаку "есть заказ из другого штата". Есть ли зависимость?

In [ ]:
another_state_days = pd.merge(pd.merge(orders[['order_id', 'customer_id', 'order_approved_at', 'order_delivered_customer_date']],
                                       order_customer[['customer_id', 'customer_state']]),
                              pd.merge(order_items[['order_id', 'seller_id', 'price', 'freight_value']],
                        sellers[['seller_id', 'seller_state']])).dropna()
another_state_days['costs'] = another_state_days['price'] + another_state_days['freight_value']
another_state_days['is_another_state'] = another_state_days['customer_state'] != another_state_days['seller_state']
another_state_days['order_approved_at'] = another_state_days['order_approved_at'].apply(lambda x: parse(x))
another_state_days['order_delivered_customer_date'] = another_state_days['order_delivered_customer_date'].apply(lambda x: parse(x))
another_state_days['days']= another_state_days['order_delivered_customer_date'] - another_state_days['order_approved_at']
another_state_days['days']=another_state_days['days'].apply(lambda x:x.days)
another_state_days=another_state_days.drop(another_state_days[another_state_days['days'] < 0].index)
fig=px.bar(another_state_days.groupby(['days', 'is_another_state'])['costs'].mean().reset_index(), x='days', y='costs', color='is_another_state', title='The price of waiting for a package', height=750)
fig.show()

Из графика можно заметить, что товары, которые отправляются в другой штат, стоят немного дороже. Происходит это потому что цена за доставку в другой штат выше, чем по штату.
Ещё видно, что красных полосочек много в правой части графика. Это значит, что товары, идущие в другой штат, доставляются медленне, чем по штату.

И первая, и вторая зависимости более чем логичны и жизненны.

## Задание 19 (0.6 баллов)

Отобразите на карте все точки продавцов (если они находятся в одном месте, то необходимо отметить кол-во)

Где больше всего продавцов находится?

In [5]:
qawa=geolocation.groupby(['geolocation_zip_code_prefix','geolocation_city']).agg({'geolocation_lat':'mean','geolocation_lng':'mean'}).reset_index().drop_duplicates()

seller_locations = pd.merge(sellers[['seller_id', 'seller_zip_code_prefix']], qawa[
    ['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city']].rename(
    columns={'geolocation_zip_code_prefix': 'seller_zip_code_prefix'}))

fig=go.Figure(go.Scattermapbox(lat=seller_locations['geolocation_lat'],lon=seller_locations['geolocation_lng'],text=seller_locations['geolocation_city']))
fig.update_layout(mapbox_style="open-street-map")
fig.show()

Как видно, больше всего продавцов находится в Сан-Паулу и Рио-де-Женейро, что в целом логично, раз это крупнейшие города Бразилии.

## Задание 20 (1,5 балл)

Определите топ-5 продавцов, которые чаще всего отсылают свою посылку в другие регионы и визуализируйте их отсылки

In [ ]:
#сначала мёржим штаты отправки и штаты доставки, если они разные, то ставим 1, иначе 0; просуммировав, у кого больше чиселка, тот и чаще других
#отправляет заказы в другие штаты

send_other_state = pd.merge(
    pd.merge(pd.merge(order_items[['order_id', 'seller_id', 'product_id']], sellers[['seller_id', 'seller_state']]),
             orders[['order_id', 'customer_id']]),
    order_customer[['customer_id', 'customer_unique_id', 'customer_state']]).drop_duplicates()
send_other_state['check'] = (send_other_state['seller_state'] != send_other_state['customer_state']).astype(int)
sellers_other_state = pd.merge(pd.merge(pd.merge(
    pd.merge(send_other_state.groupby('seller_id')['check'].sum().reset_index().sort_values('check', ascending=False)[
                 'seller_id'][:5], order_items[['seller_id', 'order_id']]), orders[['order_id', 'customer_id']]),
    sellers[['seller_id', 'seller_zip_code_prefix']]), order_customer[['customer_id', 'customer_zip_code_prefix']])

locations = geolocation[['geolocation_zip_code_prefix', 'geolocation_state']].drop_duplicates()

sellers_other_state = pd.merge(sellers_other_state, locations, left_on='seller_zip_code_prefix',
                               right_on='geolocation_zip_code_prefix').drop('geolocation_zip_code_prefix',
                                                                            axis=1).rename(
    columns={'geolocation_state': 'from'})

sellers_other_state = pd.merge(sellers_other_state, locations, left_on='customer_zip_code_prefix',
                               right_on='geolocation_zip_code_prefix').drop('geolocation_zip_code_prefix',
                                                                            axis=1).rename(
    columns={'geolocation_state': 'to'})

sellers_other_state = sellers_other_state.drop(
    ['order_id', 'customer_id', 'seller_zip_code_prefix', 'customer_zip_code_prefix'], axis=1)


                               
''' Может быть у вас получится это запустить, но я не смог, потому что у меня лёг колаб от этого)))'''

#flat = list(sellers_other_state['from_lat'])
# flon = list(sellers_other_state['from_lng'])
# tlat = list(sellers_other_state['to_lat'])
# tlon = list(sellers_other_state['to_lat'])
                               
# fig = go.Figure(go.Scattermapbox(lat=list(sellers_other_state['from_lat'])+list(sellers_other_state['to_lat']),lon=list(sellers_other_state['from_lng'])+list(sellers_other_state['to_lng'])))
# for i in range(len(flat)):
#     fig.add_trace(go.Scattermapbox(mode="lines",
#                                    hoverinfo='skip',
#                                    lat=[flat[i], tlat[i]],
#                                    lon=[flon[i], tlon[i]]))
# fig.update_layout(mapbox_style="open-street-map")
# fig.show()


'''Поэтому пойдём другим путём: нам ведь фактически нужны просто пересылки из одного штата в другой. Поэтому для каждого штата возьмём среднюю точку, 
куда приходят заказы, и будет туда проводить линии. Чтобы они не накладовались друг на друга, добавим небольшого рандомного шума к координатам; также 
каждому селлеру дадим конкретный цвет, чтобы было понятно, что это именно он. Ещё, чтобы было понятнее, по какому пути прошло больше пересылок, увеличим
толщину линии'''
                               
 

las = list(sellers.seller_state.unique())

ans = sellers_other_state[sellers_other_state['to'] == las[0]].groupby('seller_id')[
    'to'].count().reset_index().rename(columns={'to': las[0]})
for i, state in enumerate(las):
    if i != 0:
        ans = pd.merge(ans, sellers_other_state[sellers_other_state['to'] == state].groupby('seller_id')[
            'to'].count().reset_index().rename(columns={'to': state}), how='outer')
sellers5_state = sellers_other_state[['seller_id', 'from']].drop_duplicates()
sellers5_state = dict(zip(list(sellers5_state['seller_id']), list(sellers5_state['from'])))
ans=ans.fillna(0)


locat = geolocation.groupby('geolocation_state').agg(
    {'geolocation_lat': 'mean', 'geolocation_lng': 'mean'}).reset_index()

fig = go.Figure(go.Scattermapbox(lat=locat['geolocation_lat'],
                                 lon=locat['geolocation_lng'],text=locat['geolocation_state']))


for i, seller in enumerate(sellers5_state):
    for state in list(las):
      if sellers5_state[seller]!=state:
        fig.add_trace(go.Scattermapbox(mode="lines+markers+text",
                                       hoverinfo='skip',
                                       lat=locat[locat['geolocation_state'].isin([sellers5_state[seller], state])][
                                               'geolocation_lat'] + random.uniform(0.01, 0.5),
                                       lon=locat[locat['geolocation_state'].isin([sellers5_state[seller], state])][
                                               'geolocation_lng'] + random.uniform(0.01, 0.5),
                                       name=f"from {sellers5_state[seller]} to {state} by {seller}",
                                       text=[str(ans.at[i, state])]*10,
                                       textposition="bottom center", line={'color':px.colors.qualitative.G10[i],'width':ans.at[i, state]//50+1}))


fig.update_layout(mapbox_style="open-street-map", height=1000,mapbox_zoom=3, mapbox_center_lat = -25,mapbox_center_lon=-45)
fig.show()

## Задание 21* (2 балла)

Сделайте анимацию по датам с доставкой между продавцами и пользователями

## Задание 22* (творческое, отдельно поставим баллы)

Проведите дополнительно исследование между параметрами, которые вам интересны и могут быть полезными (обоснуйте, зачем это смотреть). Сделайте выводы

**Прекрасная аналогия c нами**(Они очень хорошо и быстро летают, но ходят максимально неуклюже)

## Попугай для ДЗ

![](https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Rose-ringed_parakeet_%28Psittacula_krameri_manillensis%29.jpg/1024px-Rose-ringed_parakeet_%28Psittacula_krameri_manillensis%29.jpg)

А это ожереловый попугай (или индийский кольчатый попугай). Этот вид явно выделяет наличие такого колечка на шее. Самый распространенный попугай в мире

Они очень хорошо и быстро летают, но ходят максимально неуклюже)

В отличии от многих попугаев, для которых человеческая деятельность является вредом и уменьшает их естественный ареал обитания, то как раз для этих птиц человеческая деятельность - это плюс, потому что они находят корм внутри человеческих обществ (не боятся людей, лучше всего приспосабливаются, едят пшеницу). Поговаривают, что в качестве домашних попугаев их держали еще в древней Греции и Риме, а в  средневековой Индии ожереловые попугаи часто были питомцами знатных особ и монархов, причём владение этим попугаем считалось элементом роскоши и престижа